# Raw read processing

Trim adaptors from raw reads 

In [ ]:
#!/bin/bash -e

#SBATCH --job-name=rawdata_hihi_wgs_trim.sl
#SBATCH --account=project
#SBATCH --time=00-12:00:00
#SBATCH --mem=5GB
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=16
#SBATCH --array=1-31

module load TrimGalore/0.6.7-gimkl-2020a-Python-3.8.2-Perl-5.30.1

FILE=$(sed "${SLURM_ARRAY_TASK_ID}q;d" /nesi/nobackup/project/project_1/Nc2_HihiWGS/data/sequencing_file_ids.txt)

echo "working with sample:" $FILE

OUTPUT_DIR=/nesi/nobackup/project/project_1/Nc2_HihiWGS/data/trimmed
RAW_DATA_R1=/nesi/nobackup/project/Hihi_2022/WGS_low_coverage_2022/${FILE}_R1_001.fastq.gz
RAW_DATA_R2=/nesi/nobackup/project/Hihi_2022/WGS_low_coverage_2022/${FILE}_R2_001.fastq.gz

trim_galore -j 16 -o ${OUTPUT_DIR} --fastqc --paired ${RAW_DATA_R1} ${RAW_DATA_R2}

Merge reads across lanes for each individual

In [ ]:
#!/bin/bash -e
 
#SBATCH --job-name=rawdata_hihi_wgs_trim-merge.sl
#SBATCH --account=project
#SBATCH --time=20:00:00
#SBATCH --mem=20GB
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=16

cd /nesi/nobackup/project/project_1/Nc2_HihiWGS/data

ls -lh /nesi/nobackup/project/Hihi_2022/WGS_low_coverage_2022 | awk '{print $9}' | grep "R1" | sed 's/_/\t/g' | cut -f1 | grep -v
"Undetermined\|Blank\" | sort | uniq > sample_file_ids.txt

for SAMPLE in $(cat sample_file_ids.txt)
do
cat /nesi/nobackup/project/project_1/Nc2_HihiWGS/data/trimmed/${SAMPLE}*_1.fq.gz >
/nesi/nobackup/project/project_1/Nc2_HihiWGS/data/trimmed_merged/${SAMPLE}_R1.fq.gz
cat /nesi/nobackup/project/project_1/Nc2_HihiWGS/data/trimmed/${SAMPLE}*_2.fq.gz >
/nesi/nobackup/project/project_1/Nc2_HihiWGS/data/trimmed_merged/${SAMPLE}_R2.fq.gz
done

Index reference genome

In [ ]:
#!/bin/bash -e

#SBATCH --job-name=VarCalling_index_hihi.sl
#SBATCH --account=project
#SBATCH --time=00-02:00:00
#SBATCH --mem=5GB
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=2

cd /nesi/nobackup/project/project_1/Nc2_HihiWGS/data/resources

module load BWA/0.7.17-GCC-9.2.0

GENOME=/nesi/nobackup/project/Hihi_polished_genome/AGDR_data_bundle/scaffolded_genome.fa.gz
cp $GENOME .
gunzip scaffolded_genome.fa.gz
bwa index scaffolded_genome.fa

Align reads to reference genome

In [ ]:
#!/bin/bash -e

#SBATCH --job-name=VarCalling_hihi_wgs_map.sl
#SBATCH --account=project
#SBATCH --time=00-12:00:00
#SBATCH --mem=20GB
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=16
#SBATCH --array=1-31

# load modules
module load BWA/0.7.17-GCC-9.2.0
module load SAMtools/1.15.1-GCC-11.3.0

cd /nesi/nobackup/project/project_1/Nc2_HihiWGS/data/mapped_reads

# set paths
SAMPLE=$(sed "${SLURM_ARRAY_TASK_ID}q;d" /nesi/nobackup/project/project_1/Nc2_HihiWGS/data/sample_file_ids.txt)
echo "working with sample:" $SAMPLE
GENOME=/nesi/nobackup/project/project_1/Nc2_HihiWGS/data/resources/scaffolded_genome.fa
TRIM_DATA_R1=/nesi/nobackup/project/project_1/Nc2_HihiWGS/data/trimmed_merged/${SAMPLE}_R1.fq.gz
TRIM_DATA_R2=/nesi/nobackup/project/project_1/Nc2_HihiWGS/data/trimmed_merged/${SAMPLE}_R2.fq.gz
OUT_DIR=/nesi/nobackup/project/project_1/Nc2_HihiWGS/data/mapped_reads/

# Map the reads
bwa mem -t ${SLURM_CPUS_PER_TASK} \
-R "@RG\tID:${SAMPLE}\tLB:${SAMPLE}_WGS\tPL:ILLUMINA\tSM:${SAMPLE}" \
-M ${GENOME} ${TRIM_DATA_R1} ${TRIM_DATA_R2} | \
samtools sort | samtools view -O BAM -o ${OUT_DIR}/${SAMPLE}.sorted.bam

# Check output
samtools flagstat ${OUT_DIR}/${SAMPLE}.sorted.bam

Identify duplicate reads

In [ ]:
#!/bin/bash -e

#SBATCH --job-name=VarCalling_hihi_wgs_dup.sl
#SBATCH --account=project
#SBATCH --time=00-12:00:00
#SBATCH --mem=60GB
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=2
#SBATCH --array=1-31

# load modules
module load picard/2.26.10-Java-11.0.4
module load SAMtools/1.15.1-GCC-11.3.0

# set paths
SAMPLE=$(sed "${SLURM_ARRAY_TASK_ID}q;d" /nesi/nobackup/project/project_1/Nc2_HihiWGS/data/sample_file_ids.txt)
echo "working with sample:" $SAMPLE
OUT_DIR=/nesi/nobackup/project/project_1/Nc2_HihiWGS/data/mapped_reads
TMPDIR=/nesi/nobackup/project/project_1/Nc2_HihiWGS/data/mapped_reads/tmp
export _JAVA_OPTIONS=-Djava.io.tmpdir=${TMPDIR}

# Mark Duplicates
picard MarkDuplicates INPUT=${OUT_DIR}/${SAMPLE}.sorted.bam OUTPUT=${OUT_DIR}/${SAMPLE}.sorted.dup.bam
METRICS_FILE=${OUT_DIR}/${SAMPLE}.metrics.txt MAX_FILE_HANDLES_FOR_READ_ENDS_MAP=1000;

# Generate index
samtools index -@ ${SLURM_CPUS_PER_TASK} ${OUT_DIR}/${SAMPLE}.sorted.dup.bam

# SNP calling

In [ ]:
#!/bin/bash -e
 
#SBATCH --job-name=VarCalling_hihi_wgs_mpileup.sl
#SBATCH --account=project
#SBATCH --time=00-72:00:00
#SBATCH --mem=6GB
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=2

# load modules
module load BCFtools/1.13-GCC-9.2.0
module load SAMtools/1.15.1-GCC-11.3.0

# create file where BAM files list will go
OUT_DIR=/nesi/nobackup/project/project_1/Nc2_HihiWGS/data/snp_variants_updated/

touch ${OUT_DIR}/sample_bamfiles_list.txt

# Fill BAM file list
for SAMPLE_NUMBER in {1..31}
do
SAMPLE=$(sed "${SAMPLE_NUMBER}q;d"  /nesi/nobackup/project/project_1/Nc2_HihiWGS/data/sample_file_ids.txt )
DIR=/nesi/nobackup/project/project_1/Nc2_HihiWGS/data/mapped_reads
BAM=${DIR}/${SAMPLE}.sorted.dup.bam
echo ${BAM} >> ${OUT_DIR}/sample_bamfiles_list.txt
done

# set paths
GENOME=/nesi/nobackup/project/project_1/Nc2_HihiWGS/data/resources/scaffolded_genome.fa
BAM_LIST=${OUT_DIR}/sample_bamfiles_list.txt

## Calling variants
bcftools mpileup -a 'DP,AD,ADF,ADR,SP' -Ou -f ${GENOME} -b ${BAM_LIST} | \
bcftools call -c | \
bcftools view --exclude-types indels | \
bcftools sort --temp-dir ${OUT_DIR}/temp -Oz -o ${OUT_DIR}/Hihi_wgs.vcf.gz

# SNP filtering

retain SNPs with a genotype depth of at least 5, on autosomal chromosomes and with no missingness, remove non-variant sites, singletons, and multi-allelic SNPs

In [ ]:
#!/bin/bash -e

#SBATCH --job-name      ref-filter_site
#SBATCH --mem           10G
#SBATCH --time          5:00:00
#SBATCH --cpus-per-task=12
#SBATCH --account       project

# set up environment
module load PLINK/2.00a2.3
module load BCFtools/1.19-GCC-11.3.0
module load SAMtools/1.19-GCC-12.3.0
module load VCFtools/0.1.15-GCC-9.2.0-Perl-5.30.1

# set working directory 1
cd /nesi/nobackup/project/project_1/Nc2_HihiWGS/data/snp_variants_updated/

# filter sample and depth
vcftools --gzvcf Hihi_wgs.vcf.gz --remove remove-sample.txt --mac 1 --minDP 5 --max-meanDP 100 --recode --recode-INFO-all --out hihi_wgs_filter_highcov

# set working directory 2
maindir=/nesi/nobackup/project/project_2/imputation-input/ref
cd $maindir

# copy file
cp /nesi/nobackup/project/project_1/Nc2_HihiWGS/data/snp_variants_updated/hihi_wgs_filter_highcov.recode.vcf .

# retain placed contigs
bcftools view hihi_wgs_filter_highcov.recode.vcf \
--regions 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,1A,25A,25B,4A,LGE22 \
-Oz -o hihi_wgs_filter_highcov-autosomes.vcf.gz

# rename SNP ID
bcftools annotate --set-id +'%CHROM\_%POS' hihi_wgs_filter_highcov-autosomes.vcf.gz \
-Oz -o hihi_wgs_filter_highcov-autosomes-snpID.vcf.gz

# basic stats
bcftools stats -s "-" hihi_wgs_filter_highcov-autosomes-snpID.vcf.gz > hihi_wgs_filter_highcov-autosomes-snpID.stats

# calculate missingness 
plink2 --vcf hihi_wgs_filter_highcov-autosomes-snpID.vcf.gz --missing --chr-set 95 --allow-extra-chr --out hihi_wgs_filter_highcov-autosomes-snpID

# remove variants with missingness > threshold
bcftools view -i 'F_MISSING=0' hihi_wgs_filter_highcov-autosomes-snpID.vcf.gz -Oz -o hihi_wgs_filter_highcov-autosomes-snpID-noMissing.vcf.gz

# remove non-variants & singletons & multiallelic SNPs
bcftools view -Oz -o hihi_wgs_filter_highcov-autosomes-snpID-noMissing-noSingleton-2allele.vcf.gz -i 'AC>1' --max-alleles 2 hihi_wgs_filter_highcov-autosomes-snpID-noMissing.vcf.gz

tabix -f hihi_wgs_filter_highcov-autosomes-snpID-noMissing-noSingleton-2allele.vcf.gz

bcftools stats -s "-" hihi_wgs_filter_highcov-autosomes-snpID-noMissing-noSingleton-2allele.vcf.gz > hihi_wgs_filter_highcov-autosomes-snpID-noMissing-noSingleton-2allele.stats

# calculate missingness 
plink2 --vcf hihi_wgs_filter_highcov-autosomes-snpID-noMissing-noSingleton-2allele.vcf.gz --missing --chr-set 95 --allow-extra-chr --out hihi_wgs_filter_highcov-autosomes-snpID-noMissing-noSingleton-2allele